In [101]:
import requests
from bs4 import BeautifulSoup
URL = 'https://www.gov.uk/service-standard-reports'
assessments_pg1 = requests.get(URL)
assessments_pg1_content = BeautifulSoup(page.content, 'html.parser')


In [102]:
import re
pagination_label = assessments_pg1_content.find('span',class_ = 'gem-c-pagination__link-label').text.strip()
# do I need to add something if there is just one page? probably never happen?
final_page = re.search('.* of (.*)', pagination_label, re.IGNORECASE).group(1)


In [116]:
URL = 'https://www.gov.uk/service-standard-reports?page='
i = 1
service_details = []
while i <= int(final_page):
    page = requests.get(URL+str(i))
    content = BeautifulSoup(page.content, 'html.parser')
    results = content.find_all('li', class_='gem-c-document-list__item')
    for result in results: 
        attribute_array = {"service":[],'attributes':{"url":[],"result":[],"stage":[],"assessed":[]}}
        title_html = result.find('a', class_='gem-c-document-list__item-title gem-c-document-list__item-link')
        title_text = title_html.text.strip()
        attribute_array['service'] = title_text
        url = result.find('a')['href']
        attribute_array['attributes']['url'] = url
        attributes = result.find_all('li', class_='gem-c-document-list__attribute')
        for attribute in attributes:
            atr_text = attribute.text.strip()
            if re.match('Result: (.*)', atr_text) is not None:
                attribute = re.search('Result: (.*)', atr_text, re.IGNORECASE).group(1)
                attribute_array['attributes']['result'] = attribute 
            if re.match('Assessed: (.*)', atr_text) is not None:
                attribute = re.search('Assessed: (.*)', atr_text, re.IGNORECASE).group(1)
                attribute_array['attributes']['assessed'] = attribute
            if re.match('Stage: (.*)', atr_text) is not None:
                attribute = re.search('Stage: (.*)', atr_text, re.IGNORECASE).group(1)
                attribute_array['attributes']['stage'] = attribute
        service_details.append(attribute_array)
    i += 1